In [1]:
import threading

In [144]:
def task():
    print("hello from task, thread:", threading.get_native_id())

t = threading.Thread(target=task)
t.start()
print("hello from main, thread:", threading.get_native_id())

hello from task, thread:hello from main, thread: 264
 574


In [163]:
total = 0
def task2(count):
    global total
    for i in range(count):
        total += 1

t = threading.Thread(target=task2, args=[1_000_000])
t.start()
t.join() # wait for the thread to finish
print(total)

1000000


In [170]:
total = 0
def task3(count):
    global total
    for i in range(count):
        total += 1

t1 = threading.Thread(target=task3, args=[1_000_000])
t2 = threading.Thread(target=task3, args=[1_000_000])
t1.start()
t2.start()
t1.join() # wait for the thread to finish
t2.join()
print(total)

1182554


In [171]:
import dis

In [173]:
print(dis.dis("total += 1"))

  0           RESUME                   0

  1           LOAD_NAME                0 (total)
              LOAD_CONST               0 (1)
              BINARY_OP               13 (+=)
              STORE_NAME               0 (total)
              RETURN_CONST             1 (None)
None
